Some times more than generic searches, we would like to exploit the fact that we count with a trustful resource, a curated knowledge that the agent can leverage.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

A simplified version of the previous example using Agno and LanceDB may show the potential of higher level frameworks.

### Vector database

[LanceDB](https://www.lancedb.com/) is a simple solution for tests and trials as it offers both, Cloud hosted and on-premises OSS solutions.

Let's first select a site to load data from. For example, some site documentation https://python.langchain.com/docs/introduction/

In [7]:
from agno.vectordb.lancedb import LanceDb, SearchType
from agno.knowledge.knowledge import Knowledge
from agno.knowledge.embedder.google import GeminiEmbedder

# Database
vector_db = LanceDb(
    table_name="langchain",
    uri="./lancedb",  # You can change this path to store data elsewhere
    embedder=GeminiEmbedder(),
    use_tantivy=True
)

# Knowledge base
knowledge_base = Knowledge(
    vector_db=vector_db,
)

In [14]:
from agno.knowledge.chunking.markdown import MarkdownChunking
from agno.knowledge.reader.markdown_reader import MarkdownReader

await knowledge_base.add_content_async(
    path="../markdown_files/",
    reader=MarkdownReader(
        name="Markdown Chunking Reader",
        chunking_strategy=MarkdownChunking(),
    ),
)

INFO Loading content: fd9894fd-608b-57d6-ae42-550f31c4d6d4

INFO Adding content from path, fd9894fd-608b-57d6-ae42-550f31c4d6d4, None, ../markdown_files/, None

INFO Adding content from path, 49bea173-a38d-585e-b1fc-aba5e10cc1c9, None, ../markdown_files/agents.md, None

INFO Adding file ../markdown_files/agents.md due to include/exclude filters

INFO Reading: ../markdown_files/agents.md

WARNING  Contents DB not found for knowledge base

You can check how the embedder does its magic against the knowledge base.

In [15]:
query = {
    "text" : "What are agents?",
    "vector" : GeminiEmbedder().get_embedding("What are agents?")
}
query['vector'][:10]

[-0.025601903,
 -0.015883127,
 -0.006648923,
 -0.057227448,
 -0.007338925,
 0.021304669,
 -0.00434841,
 0.010607597,
 -0.0051602297,
 0.030773513]

Of course, some tools we might need.

In [18]:
from agno.tools.tavily import TavilyTools

# Search engine
search = TavilyTools()

Now we can equip our agent with those options.

In [20]:
from agno.agent import Agent
from agno.models.google import Gemini

# Create an Agent
agent = Agent(
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    knowledge=knowledge_base,
    tools=[search],
    # search_knowledge=True gives the Agent the ability to search on demand
    # search_knowledge is True by default
    search_knowledge=True,
    instructions=[
        "Include sources in your response.",
        "Always search your knowledge before answering the question.",
        "Only include the output in your response. No other text.",
    ],
    markdown=True
)

In [22]:
agent.print_response("What are agents?", stream=True)

Output()

INFO Found 3 documents

It is a small knowledge base but it made its magic.